<a href="https://colab.research.google.com/github/marcelozurita/mariahelpdesk/blob/main/MariaHelpDesk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MarIA HelpDesk
Este é o projeto final da Imersão IA da Alura em parceria com o Google. **MarIA HelpDesk** é um protótipo funcional de um sistema de help desk para uma empresa de tecnologia que funciona com reconhecimento de voz e gera respostas em audio.

**Autor** Marcelo Zurita (marcelozurita@gmail.com)

In [1]:
!pip install -U -q google-generativeai
!pip -q install gTTS
!pip -q install ffmpeg-python

Importações e definições iniciais

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
from gtts import gTTS # Google Text To Speech
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GTTS_LANG = 'pt'  # Código para o idioma português
GTTS_TLD = 'com.br'

# Configuração da API do Google
genai.configure(api_key=GOOGLE_API_KEY)

Listando models para embedding

In [3]:
for m in genai.list_models() :
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Definindo o model padrão para o embedding. Em uma implementação final, isso deverá ser disponibilizado como uma configuração para o usuário.

In [4]:
model = 'models/embedding-001'

Documentos que serão embutidos. Em uma implementação final, esses documentos serão lidos de uma base existente de chamados do helpdesk da empresa

In [5]:
# Listagem de documentos que serão embutidos
doc1 = {
    'Titulo': 'Sqlite.dll not found ao iniciar iManager',
    'Cliente': 'IACIL',
    'Descricao': 'Está ocorrendo um erro de Sqlite.dll not found ao iniciar iManager, MobileSync e qualquer outra aplicação que acesse o sqlite.',
    'Solucao': 'Em alguns casos anteriores, o problema foi resolvido executando sfc /scannow na linha de comando iniciada como administrador, mas mas neste caso, o problema não foi resolvido.\n' +
               'Só foi resolvido depois de reiniciar o Windows.',
    'Fonte': 'https://suportecnnt.dnsalias.net/egroupware/index.php?menuaction=tracker.uitracker.edit&tr_id=4787'
}

doc2 = {
    'Titulo': 'Unable to load sqlite.dll',
    'Cliente': 'Fikafrio',
    'Descricao': 'Depois de um tempo sem receber pedidos e alguns retornos de erro no Email, verificamos que o CnntSync estava dando um erro ao carregar a dll do SQLite: unable to load sqlite.dll. O erro poderia ser facilmente verificado ao se executar o config.bat na pasta sync\bin, mas ocorria em qualquer chamada do CnntSync.',
    'Solucao': """O erro é uma falha recorrente e normalmente resolvíamos isso reiniciando o Windows.
Pelo que li, trata-se de um erro no cache de dll do sistema operacional. Por algum motivo ele não carregava mais a dll que estava na mesma pasta do executável.
Para corrigir o problema, dessa vez, foi aberta uma linha de comando como administrador e executado o seguinte comando:
sfc /scannow
Tal comando deve restaurar o cache de dll do sistema operacional.
Opcionalmente, é possível desabilitar o cache de dll adicionando uma cahave "AlwaysUnloadDll" do tipo DWord com valor 1 no registro "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows\ CurrentVersion\Explorer" utilizando o regedit. Mas para que isso faça efeito, é preciso reiniciar o Windows.
Após o sfc /scannow e encerrar as instâncias do cnntsync na memória, o cnntsync abriu normalmente.""",
    'Fonte': 'https://suportecnnt.dnsalias.net/egroupware/index.php?menuaction=tracker.uitracker.edit&tr_id=4096'
}

doc3 = {
    'Titulo': 'Sqlite.dll not found ao executar CnntSync',
    'Cliente': 'Pacheco',
    'Descricao': 'Rafael informa que os vendedores não estão recebendo atualizações e ele não consegue baixar os pedidos. Analisando a situação, percebemos que está ocorrendo o erro "sqlite.dll not found" sempre que executamos o CnntSync.',
    'Solucao': 'No momento não era possível reiniciar o servidor que era a única solução que resolveria o problema. Entretanto, o problema deixou de ocorrer depois que encerramos todas as instâncias em execução do Acrobat Reader. Provavelmente isso resolveu o problema porque o Acrobat Reader também deve usar o sqlite para armazenar informações e ele deveria estar bloqueando a dll no Windows.',
    'Fonte': None
}

documents = [doc1, doc2, doc3]

df = pd.DataFrame(documents)
df


,Titulo,Cliente,Descricao,Solucao,Fonte
0,Sqlite.dll not found ao iniciar iManager,IACIL,Está ocorrendo um erro de Sqlite.dll not found...,"Em alguns casos anteriores, o problema foi res...",https://suportecnnt.dnsalias.net/egroupware/in...
1,Unable to load sqlite.dll,Fikafrio,Depois de um tempo sem receber pedidos e algun...,O erro é uma falha recorrente e normalmente re...,https://suportecnnt.dnsalias.net/egroupware/in...
2,Sqlite.dll not found ao executar CnntSync,Pacheco,Rafael informa que os vendedores não estão rec...,No momento não era possível reiniciar o servid...,None


Função que gera um embedding de um documento no modelo

In [6]:
def embed_doc(title, customer, description, solution, font) :
  content = f'Erro: {title}\nDescrição: {description}\nSolução: {solution}\nCliente: {customer}\nFonte: {font}'
  return genai.embed_content(
      model=model,
      content=content,
      title=title,
      task_type='RETRIEVAL_DOCUMENT'
  )['embedding']


Gerando os embeddings de todos os documentos do DataFrame

In [7]:
df['Embeddings'] = df.apply(lambda row: embed_doc(
    row['Titulo'], row['Cliente'], row['Descricao'], row['Solucao'], row['Fonte']
), axis=1)
df

,Titulo,Cliente,Descricao,Solucao,Fonte,Embeddings
0,Sqlite.dll not found ao iniciar iManager,IACIL,Está ocorrendo um erro de Sqlite.dll not found...,"Em alguns casos anteriores, o problema foi res...",https://suportecnnt.dnsalias.net/egroupware/in...,"[-0.008983194, 0.029288895, -0.076187424, -0.0..."
1,Unable to load sqlite.dll,Fikafrio,Depois de um tempo sem receber pedidos e algun...,O erro é uma falha recorrente e normalmente re...,https://suportecnnt.dnsalias.net/egroupware/in...,"[0.009479894, 0.025470858, -0.065209836, -0.03..."
2,Sqlite.dll not found ao executar CnntSync,Pacheco,Rafael informa que os vendedores não estão rec...,No momento não era possível reiniciar o servid...,None,"[-0.012379153, -0.00032984966, -0.04637988, -0..."


Função que gera um embedding de uma consulta e testa qual a solução anterior que melhor se adequa a esta consulta

In [8]:
def query_embeddings(query, dataframe, model) :
  query_embed = genai.embed_content(
      model=model,
      content=query,
      task_type='RETRIEVAL_QUERY'
  )['embedding']

  # calculando a distância entre o embedding da query e os embeddings do dataframe
  scalar_dots = np.dot(np.stack(dataframe['Embeddings']), query_embed)
  index = np.argmax(scalar_dots)
  return dataframe.iloc[index]

Função que utiliza IA generativa para converter a solução de um caso similar anterior para um texto de resposta à consulta enviada.

In [9]:
def generate_response_by_similar_case(query, similar_solution) :
  prompt = f'Um usuário enviou essa pergunta: {query}\n' \
           f'O texto a seguir foi uma solução utilizada em casos anteriores. Reescreva o texto da solução anterior de forma mais informal como uma resposta à pergunta, sem adicionar informações que não façam parte do texto. Solução anterior: {similar_solution}'

  model_prompt = genai.GenerativeModel('gemini-1.0-pro')
  response = model_prompt.generate_content(prompt)

  return response.text


Função que utiliza o gTTS - Google Text to Speech para ler a resposta. Em uma implementação final, isso deve ser substituído por bibliotecas IA para uma uma leitura mais natural.

In [10]:
def speech_text(response) :
  sound_file = 'response.wav'
  tts = gTTS(response, lang=GTTS_LANG, tld=GTTS_TLD)
  tts.save(sound_file)
  return Audio(sound_file, autoplay=True)


Apenas para testar as consultas.

In [11]:
consulta = 'Está ocorrendo o erro sqlite.dll not found na hora de executar o iManager. O que fazer?'
caso_similar = query_embeddings(consulta, df, model)
solucao = caso_similar['Solucao']
fonte = caso_similar['Fonte']
resposta = generate_response_by_similar_case(consulta, solucao)
print(f'Resposta: {resposta}')
print(f'Para saber mais, consulte {fonte}')
speech_text(resposta)

Resposta: Se você estiver enfrentando o erro "sqlite.dll not found" ao executar o iManager, tente reiniciar o Windows. Em alguns casos, apenas reiniciar o computador resolveu o problema.
Para saber mais, consulte https://suportecnnt.dnsalias.net/egroupware/index.php?menuaction=tracker.uitracker.edit&tr_id=4787


In [12]:
from IPython.display import HTML, Javascript
from google.colab.output import eval_js
import base64

In [13]:
js = Javascript("""
    async function recordAudio() {
      const div = document.createElement('div');
      const audio = document.createElement('audio');
      const startButton = document.createElement('button');
      const stopButton = document.createElement('button');

      startButton.textContent = 'Gravar';
      stopButton.textContent = 'Parar';

      document.body.appendChild(div);
      div.appendChild(startButton);
      div.appendChild(audio);

      const stream = await navigator.mediaDevices.getUserMedia({audio:true});
      let recorder = new MediaRecorder(stream);

      audio.style.display = 'block';
      audio.srcObject = stream;
      audio.controls = true;
      audio.muted = true;

      await new Promise((resolve) => startButton.onclick = resolve);
        startButton.replaceWith(stopButton);
        recorder.start();

      await new Promise((resolve) => stopButton.onclick = resolve);
        recorder.stop();
        let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
        let arrBuff = await recData.data.arrayBuffer();
        stream.getAudioTracks()[0].stop();
        div.remove();

        let binaryString = '';
        let bytes = new Uint8Array(arrBuff);
        bytes.forEach((byte) => { binaryString += String.fromCharCode(byte) });

      const url = URL.createObjectURL(recData.data);
      const player = document.createElement('audio');
      player.controls = true;
      player.src = url;
      document.body.appendChild(player);

      return btoa(binaryString);

    }
""")

In [ ]:
display(js)
output = eval_js('recordAudio({})')
with open('audio.wav', 'wb') as file:
  binary = base64.b64decode(output)
  file.write(binary)

print(f'Gravação salva em { file.name }')

<IPython.core.display.Javascript object>

Gravação salva em audio.wav


In [14]:
# prompt: audio_to_blob: função recebe como parâmetro o nome de um arquivo wav e retorne um goggle.ai.generativelanguage.Blob gerado a partir do arquivo.

import google.ai.generativelanguage as glang

def audio_to_blob(filename):
  """Converts an audio file to a Google.AI.GenerativeLanguage.Blob object.

  Args:
    filename: Path to the audio file.

  Returns:
    A Google.AI.GenerativeLanguage.Blob object.
  """

  with open(filename, "rb") as f:
    file_content = f.read()

  return glang.Blob(data=file_content, mime_type="audio/wav")



In [16]:
from google.generativeai import GenerativeModel

def transcribe_audio(audio_file):
  """Transcribes an audio file using Google Generative AI.

  Args:
    audio_file: Path to the audio file in PCM WAV format.

  Returns:
    The transcribed text.
  """

  # Configure the model
  model_prompt = genai.GenerativeModel('gemini-1.5-pro-latest')

  response = model_prompt.generate_content(
    ['Transcreva este áudio', audio_to_blob(audio_file)]
  )

  # Return the transcribed text
  return response.text


In [33]:
transcribed_text = transcribe_audio('audio.wav')

print(f'Transcribed text: {transcribed_text}')


Transcribed text: Teste de reconhecimento de voz. P. 



# Interface principal
Para testar, execute o script e siga os passos:
- após ouvir a mensagem, clique em gravar e fale o problema que está ocorrendo. Exemplo: "Está ocorrendo um erro ao carregar sqlite.dll"
- clique em "Parar" para parar a gravação
- Aguarde alguns instantes e receberá a resposta

In [18]:
display(speech_text('Olá. Como posso ajudar?'))
display(js)
output = eval_js('recordAudio({})')
with open('audio.wav', 'wb') as file:
  binary = base64.b64decode(output)
  file.write(binary)

consulta = transcribe_audio(file.name)
caso_similar = query_embeddings(consulta, df, model)
solucao = caso_similar['Solucao']
fonte = caso_similar['Fonte']
resposta = generate_response_by_similar_case(consulta, solucao)
print(f'Resposta: {resposta}')
print(f'Para saber mais, consulte {fonte}')
speech_text(resposta)

<IPython.core.display.Javascript object>

Resposta: Pode ser que você esteja tendo esse erro porque o Acrobat Reader está aberto e usando a DLL do sqlite, o que impede que o CNT 5 a use. Feche todas as instâncias do Acrobat Reader e tente novamente executar o CNT 5.
Para saber mais, consulte None
